In [1]:
#%matplotlib nbagg
%matplotlib inline
%config InlineBackend.figure_format = 'pdf'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import cycle
linestyles = ['-','--','-.',':']
markerstyles = ['.', 'o', 'v', '*', 'x', 'd', '+', 's']
mark = cycle(markerstyles)
line = cycle(linestyles)
plt.style.use('grayscale')

In [36]:
def plt_apx_ratio_value(dead, den, tgt, gix, data, et='gurobi'):
    plt.figure()
    mark = cycle(markerstyles)
    line = cycle(linestyles)
    #plt.title('tgt' + str(tgt) + ' dead' + str(dead) + ' den' + str(den) + ' gix' + str(gix))
    opt = pd.Series(data.loc[(data.et == et) & (data.ntgts == tgt) & (data.dead == dead) &
                             (data.graphix == gix) & (data.den == den) &
                             (data.apxt == 'opt'), 'gameval'].item()).copy()
    for i in ['apx(dead)', 'apx(dead-spc)', 'apx(spc)', 'apx(10)', 'apx(20)', 'apx(30)']:
        temp = None
        temp = pd.Series(data.loc[(data.et == et) & (data.ntgts == tgt) &
                                  (data.dead == dead) & (data.graphix == gix) &
                                  (data.den == den) & (data.apxt == i),
                                  'gameval'].item()).copy()
        for t, v in temp.items():
            temp.at[t] = (temp.at[t] / opt.at[t] if t in opt.index else temp.at[t] / 1)
        plt.plot(temp, label=i, marker=next(mark), linestyle=next(line))
    plt.legend()
    plt.xlabel('# Resources')
    plt.ylabel('Utility Ratios')
    plt.show()
    
def plt_et_mean_value(dead, den, data, apxt='opt'):
    plt.figure()
    mark = cycle(markerstyles)
    line = cycle(linestyles)
    seldata = (data.loc[((data.dead == dead) | (data.reldead == dead)) &
                        (data.den == den) & (data.apxt == apxt),
                        ['ntgts', 'et', 'gameval']]).copy()
    seldata['gameval'] = seldata['gameval'].apply(lambda x: x.mean())
    ax=None
    seldata = seldata.groupby(['et', 'ntgts'], as_index=False).mean()
    group = seldata.groupby(['et'])
    gurobi = group.get_group('gurobi').reset_index()
    for n, g in group:
        g = g.reset_index()
        g.loc[:,'gameval'] -= gurobi.loc[:,'gameval']
        ax = g.plot(x='ntgts', y='gameval', label=n, ax=ax)
    plt.legend()
    plt.xlabel('# Nodes')
    plt.ylabel('Mean Utility')
    plt.show()

def plt_perc_time(dead, den, tgt, gix, apxt, data, et='gurobi'):
    mark = cycle(markerstyles)
    line = cycle(linestyles)
    temp = data.loc[(data.et == et) & (data.ntgts==tgt) &
                    ((data.dead==dead) | (data.reldead==dead)) &
                    (data.graphix==gix) & (data.den==den) & (data.apxt==apxt),
                    ['time_short', 'time_cov', 'time_dom', 'time_max', 'time_res', 'time_tot']]
    if not temp.size:
        print('No results exists')
        return
    plt.figure()
    mark = cycle(markerstyles)
    line = cycle(linestyles)
    #plt.title('tgt'+str(tgt)+' dead'+str(dead)+' den'+str(den)+' gix'+str(gix))
    temp['time_dom'].fillna(0, inplace=True)
    temp['time_cov'] += temp['time_dom']
    del(temp['time_dom'])
    del(temp['time_tot'])
    seriestemp = temp.reset_index().iloc[:, 1:-1].squeeze()
    for i, t in temp.time_res.item().items():
        name='time_res_'+str(i)
        seriestemp =  seriestemp.set_value(name, t)

    seriestemp /= seriestemp.sum()
    seriestemp *= 100
    seriestemp.plot.bar()
    plt.ylabel('% Time')
    plt.show()

In [3]:
data = pd.read_pickle('data_aggregates.pickle')

In [4]:
data['den'].replace(8, 6, inplace=True);

In [5]:
data.head()

,apxt,dead,den,ei,et,gameval,graphix,improves,ntgts,optres,reldead,time_cov,time_dom,time_max,time_res,time_res_noenum,time_short,time_tot,time_tot_no_cov,err
0,apx(30),NaN,6.0,10.0,ls2,1 0.555069 2 0.867642 3 1.000000 dtyp...,6.0,"{1: [(0.5302288815856873, [(1, 9)], [([(1, 156...",25.0,3,1.5,14.002273,2.906100,13.890954,1 237.597720 2 1051.069706 dtype: float64,1 23.759772 2 105.106971 dtype: float64,0.012047,1316.574930,1302.572657,0.0
1,apx(30),NaN,10.0,10.0,ls2,1 0.546538 2 0.783656 3 0.948117 4 ...,4.0,"{1: [(0.5465376700774089, [(1, 24)], [([(1, 35...",25.0,4,1.5,13.119743,3.985260,18.988849,1 278.224037 2 1558.416026 3 3773.16...,1 27.822404 2 155.841603 3 377.31686...,0.010874,5641.930608,5628.810865,0.0
2,apx(30),NaN,10.0,10.0,ls2,1 0.400087 2 0.647140 3 0.840835 4 ...,1.0,"{1: [(0.400087067518736, [(1, 20)], [([(1, 0)]...",25.0,4,1.5,10.275149,2.571195,16.152661,1 177.176124 2 832.361658 3 2295.17...,1 17.717612 2 83.236166 3 229.51782...,0.011052,3331.157129,3320.881980,0.0
3,apx(30),NaN,6.0,10.0,ls2,1 0.598502 2 0.861762 3 1.000000 dtyp...,3.0,"{1: [(0.556626436254033, [(1, 0)], [([(1, 5)],...",25.0,3,1.5,12.493290,2.975119,15.534104,1 159.555900 2 862.050547 dtype: float64,1 15.955590 2 86.205055 dtype: float64,0.011047,1049.647080,1037.153790,0.0
4,apx(30),NaN,10.0,10.0,ls2,1 0.529799 2 0.795980 3 0.904488 4 ...,8.0,"{1: [(0.5286061697306934, [(1, 6)], [([(1, 0)]...",25.0,4,1.5,11.091565,2.340009,12.515474,1 187.370606 2 1146.021165 3 2966.35...,1 18.737061 2 114.602116 3 296.63545...,0.011127,4323.367610,4312.276045,0.0


# Compute Time

In [6]:
plt.figure()
mark = cycle(markerstyles)
line = cycle(linestyles)
dead = 5
den = 6
#plt.title('dead ' +str(dead) + ' den ' + str(den))
for apx in ['opt', 'apx(dead)', 'apx(dead-spc)', 'apx(spc)', 'apx(10)', 'apx(20)', 'apx(30)']:
    temp = data.loc[(data.et == 'gurobi') & (data.dead==dead) &
                    (data.den==6) & (data.apxt==apx),
                    ['ntgts', 'time_tot']].groupby('ntgts')
    plt.plot(temp.aggregate(np.mean), label=apx, marker=next(mark), linestyle=next(line))
plt.legend()
plt.xlabel('# Nodes')
plt.ylabel('Compute Time(s)')
plt.show()

<Figure size 432x288 with 1 Axes>

In [7]:
plt.figure()
mark = cycle(markerstyles)
line = cycle(linestyles)
dead = 10
den = 6
#plt.title('dead ' +str(dead) + ' den ' + str(den))
for apx in ['opt', 'apx(dead)', 'apx(dead-spc)', 'apx(spc)', 'apx(10)', 'apx(20)', 'apx(30)']:
    temp = data.loc[(data.et == 'gurobi') & (data.dead==dead) &
                    (data.den==6) & (data.apxt==apx),
                    ['ntgts', 'time_tot']].groupby('ntgts')
    plt.plot(temp.aggregate(np.mean), label=apx, marker=next(mark), linestyle=next(line))
plt.legend()
plt.xlabel('# Nodes')
plt.ylabel('Compute Time(s)')
plt.show()

<Figure size 432x288 with 1 Axes>

In [8]:
plt_perc_time(5, 6, 50, 0, 'opt', data)

/usr/local/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:64: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


<Figure size 432x288 with 1 Axes>

In [9]:
plt_perc_time(5, 6, 50, 0, 'apx(30)', data)

/usr/local/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:64: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


<Figure size 432x288 with 1 Axes>

In [35]:
plt.figure()
mark = cycle(markerstyles)
line = cycle(linestyles)
reldead = 2
den = 6
#plt.title('reldead ' +str(reldead) + ' den ' + str(den))
apxlist = ['apx(dead)', 'apx(dead-spc)', 'apx(spc)',
           'apx(-spc)', 'apx(10)', 'apx(20)', 'apx(30)']
for apx in apxlist:
    temp = data.loc[(data.et == 'gurobi') & (data.reldead==1.5) &
                    (data.den==6) & (data.apxt==apx),
                    ['ntgts', 'time_tot']].groupby('ntgts')
    plt.plot(temp.aggregate(np.mean), label=str(apx),
             marker=next(mark), linestyle=next(line))
plt.legend()
plt.xlabel('# Nodes')
plt.ylabel('Compute Time(s)')
plt.show()

<Figure size 432x288 with 1 Axes>

In [11]:
data.loc[(data.et == 'gurobi') & (data.reldead==2) & (data.graphix==0) & (data.ntgts==50) &
         (data.den==6) & ((data.apxt=='apx(20)') | (data.apxt=='apx(10)')), :]

,apxt,dead,den,ei,et,gameval,graphix,improves,ntgts,optres,reldead,time_cov,time_dom,time_max,time_res,time_res_noenum,time_short,time_tot,time_tot_no_cov,err
2101,apx(20),NaN,6.0,10.0,gurobi,1 0.401386 2 0.642248 3 0.808170 4 ...,0.0,"{1: [(0.40138631760774235, [(1, 49)], [([(1, 4...",50.0,5,2.0,95.529868,58.916497,442.023895,1 2852.151462 2 9555.978053 3 23041...,1 285.215146 2 955.597805 3 2304.13...,0.035349,103177.296258,103081.766390,0.009699
2410,apx(10),NaN,6.0,10.0,gurobi,1 0.406622 2 0.646527 3 0.799234 4 ...,0.0,"{1: [(0.3764533302909645, [(1, 49)], [([(1, 12...",50.0,6,2.0,71.754014,30.944000,205.300810,1 1989.141470 2 8532.726359 3 20...,1 198.914147 2 853.272636 3 2091...,0.035027,206106.625835,206034.871821,0.012251


In [12]:
plt.figure()
mark = cycle(markerstyles)
line = cycle(linestyles)
ntgts = 20
den = 10
#plt.title('ntgts ' +str(ntgts) + ' den ' + str(den))
optlist = ['opt', 'apx(dead)', 'apx(dead-spc)', 'apx(spc)', 'apx(10)', 'apx(20)', 'apx(30)']
for apx in optlist:
    temp = data.loc[(data.et == 'gurobi') & (data.ntgts==20) & (data.den==10) &
                    (data.apxt==apx), ['dead', 'time_tot_no_cov']].groupby('dead')
    plt.plot(temp.aggregate(np.mean), label=str(apx),
             marker=next(mark), linestyle=next(line))
plt.legend()
plt.xlabel('Deadline')
plt.ylabel('Cmpute Time NoCov(s)')
plt.show()

<Figure size 432x288 with 1 Axes>

# Game Value

In [13]:
plt.figure()
mark = cycle(markerstyles)
line = cycle(linestyles)
dead=5
den=6
tgt=50
gix=0
#plt.title('tgt'+str(tgt)+' dead'+str(dead)+' den'+str(den)+' gix'+str(gix))
for i in optlist:
    plt.plot(data.loc[(data.et == 'gurobi') & (data.ntgts==tgt) & (data.dead==dead) &
                      (data.graphix==gix) & (data.den==den) & (data.apxt==i),
                      'gameval'].item(),
             label=str(i),  marker=next(mark), linestyle=next(line))
plt.legend()
plt.xlabel('# Resources')
plt.ylabel('Game Utility')
plt.show()

<Figure size 432x288 with 1 Axes>

In [34]:
#%pdb
plt.figure()
mark = cycle(markerstyles)
line = cycle(linestyles)
reldead=2
den=6
tgt=50
gix=0
#plt.title('tgt'+str(tgt)+' relativedead'+str(reldead)+' den'+str(den)+' gix'+str(gix))
for i in apxlist:
    plt.plot(data.loc[(data.ntgts==tgt) & (data.reldead==reldead) & (data.graphix==gix) &
                      (data.den==den) & (data.apxt==i) & (data.et == 'gurobi'),
                      'gameval'].item(),
             label=str(i), marker=next(mark), linestyle=next(line))
plt.legend()
plt.xlabel('# Resources')
plt.ylabel('Game Utility')
plt.show()

<Figure size 432x288 with 1 Axes>

In [15]:
plt.figure()
mark = cycle(markerstyles)
line = cycle(linestyles)
dead=5
den=6
tgt=40
gix=1
#plt.title('tgt'+str(tgt)+' dead'+str(dead)+' den'+str(den)+' gix'+str(gix))
for i in optlist:
    plt.plot(data.loc[(data.et == 'gurobi') & (data.ntgts==tgt) & (data.dead==dead) &
                      (data.graphix==gix) & (data.den==den) & (data.apxt==i),
                      'gameval'].item(),
             label=str(i),  marker=next(mark), linestyle=next(line))
plt.legend()
plt.xlabel('# Resources')
plt.ylabel('Game Utility')
plt.show()

<Figure size 432x288 with 1 Axes>

# Mean Error of APX

In [16]:
dead = 5
den1 = 6
den2 = 8
plt.figure()
mark = cycle(markerstyles)
line = cycle(linestyles)
for a in ['apx(dead)', 'apx(dead-spc)', 'apx(spc)', 'apx(10)', 'apx(20)', 'apx(30)']:
    tempdata = data.loc[(data.dead==dead) & ((data.den==den1)|(data.den==den2)) &
                        (data.apxt==a), ['ntgts', 'err']]
    plt.plot(tempdata.groupby(['ntgts'])['err'].mean(), label=str(a),
             marker=next(mark), linestyle=next(line))
plt.legend()
plt.xlabel('# Nodes')
plt.ylabel('Mean Error')
plt.show()

<Figure size 432x288 with 1 Axes>

In [17]:
reldead = 2
den1 = 6
plt.figure()
mark = cycle(markerstyles)
line = cycle(linestyles)
for a in ['apx(dead)', 'apx(dead-spc)', 'apx(spc)', 'apx(10)', 'apx(-spc)', 'apx(20)']:
    tempdata = data.loc[(data.reldead==reldead) & (data.den==den1) &
                        (data.apxt==a), ['ntgts', 'err']]
    plt.plot(tempdata.groupby(['ntgts'])['err'].mean(), label=str(a),
             marker=next(mark), linestyle=next(line))
plt.legend()
plt.xlabel('# Nodes')
plt.ylabel('Mean Error')
plt.show()

<Figure size 432x288 with 1 Axes>

# Enumeration Value

In [24]:
plt.figure()
mark = cycle(markerstyles)
line = cycle(linestyles)
dead=5
den=6
tgt=50
gix=0
#plt.title('tgt'+str(tgt)+' dead'+str(dead)+' den'+str(den)+' gix'+str(gix))
for e in ['gurobi', 'ls1', 'ls2']:
    plt.plot(data.loc[(data.ntgts==tgt) & (data.dead==dead) & (data.graphix==gix) &
                      (data.den==den) & (data.apxt=='opt') & (data.et==e), 'gameval'].item(),
             label='et ' + str(e), marker=next(mark), linestyle=next(line))
plt.legend()
plt.xlabel('# Resources')
plt.ylabel('Game Utility')
plt.show()

<Figure size 432x288 with 1 Axes>

In [25]:
plt.figure()
mark = cycle(markerstyles)
line = cycle(linestyles)
reldead=1.5
den=6
tgt=50
gix=0
#plt.title('tgt'+str(tgt)+' dead'+str(dead)+' den'+str(den)+' gix'+str(gix))
for e in ['gurobi', 'ls1', 'ls2']:
    plt.plot(data.loc[(data.ntgts==tgt) & (data.reldead==reldead) & (data.graphix==gix) &
                      (data.den==den) & (data.apxt=='apx(30)') & (data.et==e),
                      'gameval'].item(),
             label=str(e), marker=next(mark), linestyle=next(line))
plt.legend()
plt.xlabel('# Resources')
plt.ylabel('Game Utility')
plt.show()

<Figure size 432x288 with 1 Axes>

In [37]:
plt_et_mean_value(5, 6, data)

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 1 Axes>

In [38]:
plt_et_mean_value(1.5, 6, data, apxt='apx(30)')

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 1 Axes>

# Enumeration Time

In [22]:
plt.figure()
mark = cycle(markerstyles)
line = cycle(linestyles)
dead = 5
den = 6
#plt.title('dead ' +str(dead) + ' den ' + str(den))
for e in ['gurobi', 'ls1', 'ls2']:
    temp = data.loc[(data.dead==dead) & (data.den==den) &
                    (data.apxt == 'opt') & (data.et == e),
                    ['ntgts', 'time_tot']].groupby('ntgts')
    plt.plot(temp.aggregate(np.mean), label=e, marker=next(mark), linestyle=next(line))
plt.legend()
plt.xlabel('# Nodes')
plt.ylabel('Compute Time(s)')
plt.show()

<Figure size 432x288 with 1 Axes>

In [23]:
plt.figure()
mark = cycle(markerstyles)
line = cycle(linestyles)
reldead = 1.5
den = 6
#plt.title('dead ' +str(dead) + ' den ' + str(den))
for e in ['gurobi', 'ls1', 'ls2']:
    temp = data.loc[(data.reldead==reldead) & (data.den==den) &
                    (data.apxt == 'apx(30)') & (data.et == e),
                    ['ntgts', 'time_tot']].groupby('ntgts')
    plt.plot(temp.aggregate(np.mean), label=e, marker=next(mark), linestyle=next(line))
plt.legend()
plt.xlabel('# Nodes')
plt.ylabel('Compute Time(s)')
plt.show()

<Figure size 432x288 with 1 Axes>

In [65]:
plt.figure()
mark = cycle(markerstyles)
line = cycle(linestyles)
dead=5
den=6
tgt=50
gix=0
cost=0.08
#plt.title('tgt'+str(tgt)+' dead'+str(dead)+' den'+str(den)+' gix'+str(gix))
for cost in [0.05, 0.07, 0.1]:
    gameval = data.loc[(data.et == 'gurobi') & (data.ntgts==tgt) & (data.dead==dead) &
                       (data.graphix==gix) & (data.den==den) & (data.apxt=='opt'),
                       'gameval'].item()
    gameutils = {k: v - k*cost for k, v in gameval.items()}
    plt.plot(*zip(*sorted(gameutils.items())),
             label='cost: ' + str(cost),  marker=next(mark), linestyle=next(line))
plt.legend()
plt.ylim(ymax=1)
plt.xlabel('# Resources')
plt.ylabel('Game Revenue')
plt.show()

<Figure size 432x288 with 1 Axes>